## The Circle Game

        We're captive on the carousel of time
        We can't return we can only look
        Behind from where we came
        And go round and round and round
        In the circle game
        -- Joni Mitchell, `Circle Game`
 

Suppose you want to compare a string, character-by-character, to itself and see how many matches there are.


In [1]:
def matches(seq: str) -> int:
    """Count matches of seq with itself."""
    _matches = 0
    for chr in seq:
        if chr == chr:
            _matches += 1
    return _matches

You know the answer already: they all match, so it's just the number of characters in the string.

In [2]:
seq = "hello, world"
matches(seq) == len(seq)

True

Change `seq` above to something else and press enter if you think `"hello, world"` is special.

But what if you shift the string a little, like this?

        hello, world
         hello, world

Now, almost nothing matches -- there's only the second `l` in the original `hello`, which matches the first `l` in the shifted version, below it.  

You also have to special-case the characters at the beginning of the original and end of the shifted version, which don't have anything to compare to. You can get rid of that nuisance by treating the string as a circle, moving the letters shifted off the end around to the beginning.

        hello, world
        dhello, worl

Let's rewrite `matches()` to do just that.

In [3]:
def matches(seq: str, shift: int) -> int:
    """Count matches of seq to a circular permutation, shifted by shift."""
    matches = 0
    length = len(seq)
    for pos, chr in enumerate(seq):
        # just use modular arithmetic to find the character to compare to
        shifted_chr = seq[(pos+shift) % length]
        if chr == shifted_chr:
            matches += 1
    return matches

A shift of 0 should give the length of the string,

In [4]:
matches(seq, 0) == len(seq)

True

and a shift of 1 should just leave a single, matching `l`.

In [5]:
matches(seq, 1)

1

The rotated version of the string is `seq[shift:] + seq[:shift]`,
so we can see the matches for each rotation like this:


In [6]:
for shift in range(len(seq)):
    n_match = matches(seq, shift)
    print(f"{seq[shift:] + seq[:shift]}: {n_match}")

hello, world: 12
ello, worldh: 1
llo, worldhe: 0
lo, worldhel: 0
o, worldhell: 2
, worldhello: 1
 worldhello,: 0
worldhello, : 1
orldhello, w: 2
rldhello, wo: 0
ldhello, wor: 0
dhello, worl: 1


After it rotates half way around, we start getting the same counds in reverse.
Of course! It's a circle, so the number of matches for a shift of one to the right

    hello, world
    dhello, worl

will be the same as the number of matches for a shift of one to the left
 
    hello, world
    ello, worldh

The first `len(seq) // 2` shifts give all possible mis-alignments. Even including the original sequence aligned with itself cuts the time to run the previous example nearly in half:

In [7]:
for shift in range(len(seq)//2 + 1):
    n_match = matches(seq, shift)
    print(f"{seq[shift:] + seq[:shift]} : {n_match=}")

hello, world : n_match=12
ello, worldh : n_match=1
llo, worldhe : n_match=0
lo, worldhel : n_match=0
o, worldhell : n_match=2
, worldhello : n_match=1
 worldhello, : n_match=0


Surprisingly, the fourth mis-alignment has a pair of matches, more than the first! Its second `o`, from `world`, matches the original `o` in `hello`, and its penultimate letter, `l`, matches the `l` in the original `world`. 

        hello, world
        o, worldhell

A longer shift gives a better match.

Before going further, let's re-write `matches()` one more time to make the output more descriptive.

Begin by making a named tuple to hold the shift and the number of matches at that position.

In [8]:
from collections import namedtuple
ShiftMatch = namedtuple("ShiftMatch", ["matches", "shift"])
ShiftMatch(shift=0, matches=12)


ShiftMatch(matches=12, shift=0)

Now `matches()` becomes this:

In [9]:
def matches(seq: str, shift: int) -> int:
    """Count matches of seq to a circular permutation, shifted by shift."""
    matches = 0
    length = len(seq)
    for pos, chr in enumerate(seq):
        # just use modular arithmetic to find the character to compare to
        shifted_chr = seq[(pos+shift) % length]
        if chr == shifted_chr:
            matches += 1
    return ShiftMatch(matches=matches, shift=shift)

seq = "hello, world"
matches(seq, 0)

ShiftMatch(matches=12, shift=0)

This tweak makes it easy to report all ShiftMatch values.

In [10]:
def shift_match_list(seq: str) -> list[ShiftMatch]:
    """Return number of matches of sequence with each possible shift of itself, sorted best to worst."""
    _shift_matches = []
    for shift in range(len(seq)//2 + 1):
        _shift_matches.append(matches(seq, shift))
    return _shift_matches

shift_match_list(seq)

[ShiftMatch(matches=12, shift=0),
 ShiftMatch(matches=1, shift=1),
 ShiftMatch(matches=0, shift=2),
 ShiftMatch(matches=0, shift=3),
 ShiftMatch(matches=2, shift=4),
 ShiftMatch(matches=1, shift=5),
 ShiftMatch(matches=0, shift=6)]

Or the same list sorted by # of matches.

In [11]:
def most_matches(seq: str) -> list[ShiftMatch]:
    return sorted(shift_match_list(seq), reverse=True)

In [12]:
most_matches(seq)

[ShiftMatch(matches=12, shift=0),
 ShiftMatch(matches=2, shift=4),
 ShiftMatch(matches=1, shift=5),
 ShiftMatch(matches=1, shift=1),
 ShiftMatch(matches=0, shift=6),
 ShiftMatch(matches=0, shift=3),
 ShiftMatch(matches=0, shift=2)]

As we've seen already, the most matches are with the unshifted string itself (no surprise), the second most are at a shift of 4.
Shifts of 1 and 5 both find only one match, and the rest have none. 

## Circular DNA

Well, okay, but so what? In the first place, who wants to compare strings with their shifted versions,
and in the second who ever has circular strings?

Let's start with the latter.
The DNA in your chromosomes is a linear string of the letters 'A', 'C', 'G', and 'T'.
Of course, these letters are chemicals -- Adenine, Cytosine, Guanine, and Thymine -- but every letter in this notebook
is really just a bit pattern, and even if you think of bits ones and zeros, they're implemented in electronics.

So. DNA is a chemical implementation of a string. (Biologists call the four letters ***bases***, 
so we'll use that language from here on.) Modern DNA-sequencing techniques supply a surfeit of such strings.

But not all DNA in your cells is in the nucleus. There's also DNA in your mitochondria. 
And while the DNA in chromosomes is linear strings in a four-letter alphabet, 
the DNA in the mitochondria of those cells is circular.

It's also a lot smaller. The biggest human chromosome has almost 250 million letters, the smallest, only about 45 million.
Human mitochondrial DNA? 16,569 bases, smaller than a 20KB file.  An old-fashioned, 80x24 terminal window shows 1920 characters, 
so only about eight of those. 

The genome of *Escherischia coli*, the workhorse of bacterial genetics, is bigger but still only about 50MB. And circular.
Bacteriophage *lambda*, a common molecular biology tool, infects *E. coli*, and is only about 50KB, a tenth the size. 
(A "bacteriophage", or "phage" is a virus that infects bacteria.)
Another *E. coli* predator, *phiX174*, is only 5K.  All three are circular. 

Circular genomes are the rule in the bacterial world, not the exception.
We have tons of strings available to us that are circular. They're all DNA.




## A Look at PhiX DNA.

Let's try our code on [the phiX174 genome](https://www.ncbi.nlm.nih.gov/nuccore/9626372). You can download it to a file by clicking on **FASTA**, then **Send to:**.


The FASTA file, `sequence.fasta`, is just a text file -- a single-line header followed by the DNA sequence, broken into 72-column lines.

In [13]:
with open("sequence.fasta") as fasta:
   lines = fasta.readlines()
print(lines[:2])

['>NC_001422.1 Escherichia phage phiX174, complete genome\n', 'GAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCGGATATTTCTGATGAGTCGAAAAATTATCTT\n']


You could parse it yourself and concatenate all the DNA fragments into a single string,
but BioPython will do that work for you.

In [14]:
import Bio.SeqIO  # the BioPython module, for handling DNA sequences.

record = Bio.SeqIO.read("sequence.fasta", "fasta")
record.description


'NC_001422.1 Escherichia phage phiX174, complete genome'

In [15]:
len(record.seq)


5386

PhiX has 5386 bases.

Let's wrap getting the sequence into a function.

In [16]:
def get_dna_sequence(filename: str) -> str:
    """Return sequence in FASTA file as a single string."""
    return Bio.SeqIO.read(filename, "fasta").seq


### Compare phiX to Itself, Shifted

Alright-y, then! What happens when we look at shift-matches for the PhiX circular genome?
Once again, the most matches will be for the un-shifted genome, but what comes next?

In [17]:
phi_x = get_dna_sequence("sequence.fasta")
sorted_matches = most_matches(phi_x)
print("best: ", sorted_matches[:2])
print("worst: ", sorted_matches[-2:])

best:  [ShiftMatch(matches=5386, shift=0), ShiftMatch(matches=1609, shift=66)]
worst:  [ShiftMatch(matches=1239, shift=2417), ShiftMatch(matches=1223, shift=478)]


The most matches, 5386, are the whole sequence. When you don't shift at all, everything matches itself.
The second-most matches, 1609, are at a shift of 66. 

Is 66 a lot? We'll have to do work to know, so let's defer that for a second and focus on the shifts instead.

In [18]:
shifts_with_most_matches = [shift_match.shift for shift_match in sorted_matches]
shifts_with_most_matches[:30]  # the 35 positions with the most matches

[0,
 66,
 78,
 9,
 12,
 57,
 21,
 96,
 48,
 33,
 30,
 180,
 87,
 3,
 132,
 195,
 102,
 6,
 39,
 51,
 156,
 15,
 120,
 90,
 45,
 249,
 93,
 18,
 69,
 126]

What do these have in common? They're all divisible by 3.
How far down the list do you need to go to find one that isn't?

In [19]:
# find the ranks of shifts indivisible by 3
i_b_3 = [sorted_matches.index(shift_match) for shift_match in sorted_matches if shift_match.shift % 3 != 0]
print(f"first indivisible by 3: {i_b_3[0]}, shift_match = {sorted_matches[i_b_3[0]]}")

first indivisible by 3: 65, shift_match = ShiftMatch(matches=1502, shift=1828)


The first shift indivisible by 3 has the 66th most matches: only 1502 out of 5386 characters.

There are 2694 -- 5386//2 + 1 -- possible shifts. Only a third of those, 898, are divisible by 3.
The 65 shifts with the most matches are in that third. (Array indices start at 0.)

There's something special about multiples of 3 in PhiX DNA.





## A Little Statistics

How many matches would you expect to see for a shift if bases were arranged at random? 


Well, there are four bases, so on average, a quarter of the letters would end up randomly matching: 
5386/4 = 1347.

Right?

Wrong.

The letters may not occur 1/4 of the time.  If, for example, you had a circular sequence that was all 'A', every shift would still match at every letter. To figure out how many matches we expect at random requires first finding out the letter distribution, or *base composition*, and then doing some math. 

Here's the base composition of PhiX

In [20]:
from collections import Counter

def base_composition(seq: str):
    composition = Counter(seq)
    return composition

base_composition(phi_x)

Counter({'G': 1254, 'A': 1291, 'T': 1684, 'C': 1157})

So the probability of two random bases matching drawn (with replacement) from these is
P('G')*P('G') + P('A')*P('A') + P('T')*P('T') + P('C')*P('C')

In [21]:
def p_random_match(seq:str) -> float:
    """ Probability of a random match.

      Probability for two bases drawn at random
      from a distribution matching seq.
    """  
    counts = base_composition(seq)
    total = len(seq)
    base_probabilities = {base: count/total for base, count in counts.items()}
    p = 0  # probability of a random match
    for base, prob in base_probabilities.items():
        p += prob*prob
    return p



For a sequence of all 'A's, every shift matches.

In [22]:
p_random_match("A"*5386)  # a phiX-length sequence of all 'A's


1.0

In [23]:
p = p_random_match(phi_x)
print(f"P(random match) = {p:2.2}, Expected total matches = {round(p*len(phi_x))}")

P(random match) = 0.26, Expected total matches = 1376


Answering our earlier question, we expect about 26% of the bases to match if phiX bases are just randomly arranged--about 1376.
The standard deviation on that last is roughly `sqrt(n*p(1-p))`.

In [24]:
from math import sqrt
sigma = sqrt(len(phi_x)*p*(1-p))
print("standard deviation =", round(sigma))

standard deviation = 32


In [26]:
d_b_3 = [shift_match for shift_match in sorted_matches if shift_match.shift % 3 == 0]
big = [shift_match for shift_match in d_b_3 if shift_match.matches > p + 3*sigma]
len(big)

898

## Simulating the Combinatorics Problem

Let's begin with a random DNA sequence with a given base composition.


In [27]:

from random import shuffle
def random_seq(composition: dict[str: int]):
    """Return a random sequence with a given composition"""
    seq = []
    for base, count in composition.items():
        seq += [base] * count
        shuffle(seq)
    return ''.join(seq)


# 5 random sequences with the same composition
for _ in range(5):
    print(random_seq({'A': 2, 'C': 3, 'G': 4, 'T': 5}))



CGTTCGATACGTGT
CCTGCTAGGGTATT
ATGCTCATGGGCTT
GCTGCGCTTATGTA
TGGCATTGTTCGAC


Now look at all the self_matches of that random sequence.

In [28]:
seq = random_seq({'A': 2, 'C': 3, 'G': 4, 'T': 5})
shift_matches = shift_match_list(seq)
shift_matches


[ShiftMatch(matches=14, shift=0),
 ShiftMatch(matches=3, shift=1),
 ShiftMatch(matches=6, shift=2),
 ShiftMatch(matches=3, shift=3),
 ShiftMatch(matches=2, shift=4),
 ShiftMatch(matches=3, shift=5),
 ShiftMatch(matches=2, shift=6),
 ShiftMatch(matches=2, shift=7)]

In [ ]:
from statistics import median

for shift_match in shift_matches:
    print(shift_match.matches)

median([shift_match.matches for shift_match in shift_matches])

14
6
2
0
2
3
5
4


3.5

Let's wrap that into a function or two:


In [31]:
from statistics import median

def median_matches(seq):
    return median([shift_match.matches for shift_match in shift_match_list(seq)])

def median_matches_random(seq: str):
    counts = base_composition(seq)
    seq = random_seq(counts)
    return median_matches(seq)

In [32]:
%time median_matches(phi_x)

CPU times: user 4.13 s, sys: 37.3 ms, total: 4.17 s
Wall time: 4.17 s


1374.0

In [ ]:
for _ in range(10):
    print(median_matches_random(phi_x))

1376.0
1375.0
1376.0
1375.0
1375.0
1376.0
1376.0
1375.5
1376.0
1376.0


The median matches for phiX174 matches a random sequence with the same composition.

But is there the same bias toward shifts of 3?

In [34]:
def indivisible_by_3(seq):
    sorted_matches = most_matches(seq)
    return [sorted_matches.index(shift_match) for shift_match in sorted_matches if shift_match.shift % 3 != 0]

l = indivisible_by_3(phi_x)
print(f"phi_x, first shift indivisible by 3: {l[0]}, shift_match = {sorted_matches[l[0]]}")
counts = base_composition(phi_x)
seq = random_seq(counts)
l = indivisible_by_3(random_seq(counts))
print(f"random sequence, phi_x composition, first shift indivisible by 3: {l[0]}, shift_match = {sorted_matches[l[0]]}")


phi_x, first shift indivisible by 3: 65, shift_match = ShiftMatch(matches=1502, shift=1828)
random sequence, phi_x composition, first shift indivisible by 3: 2, shift_match = ShiftMatch(matches=1608, shift=78)
